In [45]:
#@title Copyright 2019 Google LLC. { display-mode: "form" }
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

# Earth Engine Python API Colab Setup

This notebook demonstrates how to setup the Earth Engine Python API in Colab and provides several examples of how to print and visualize Earth Engine processed data.

## Import API and get credentials

The Earth Engine API is installed by default in Google Colaboratory so requires only importing and authenticating. These steps must be completed for each new Colab session, if you restart your Colab kernel, or if your Colab virtual machine is recycled due to inactivity.

### Import the API

Run the following cell to import the API into your session.

In [46]:
import ee

### Authenticate and initialize

Run the `ee.Authenticate` function to authenticate your access to Earth Engine servers and `ee.Initialize` to initialize it. Upon running the following cell you'll be asked to grant Earth Engine access to your Google account. Follow the instructions printed to the cell.

In [47]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='gee-481701')

## Test the API

Test the API by printing the elevation of Mount Everest.

In [48]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [50]:
boston_points = [
    (-71.04113915351529, 42.37290195969027),  # Example point 1
    (-71.04113915351529, 42.34811103210867),  # Example point 2
    (-71.08350242735263, 42.34811103210867),  # Example point 3
    (-71.08350242735263, 42.37290195969027),  # Example point 4
]

In [63]:
polygon = ee.Geometry.Polygon([boston_points])

In [68]:
buffered_region = polygon.buffer(5)

In [69]:
buffered_region

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.buffer",
    "arguments": {
      "distance": {
        "constantValue": 5
      },
      "geometry": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    -71.04113915351529,
                    42.37290195969027
                  ],
                  [
                    -71.04113915351529,
                    42.34811103210867
                  ],
                  [
                    -71.08350242735263,
                    42.34811103210867
                  ],
                  [
                    -71.08350242735263,
                    42.37290195969027
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constantValue": true
            }
          }
        }
      }
    }
  }
})

In [70]:
landsat_collections = {
    "Landsat_4": "LANDSAT/LT04/C02/T1_L2",
    "Landsat_5": "LANDSAT/LT05/C02/T1_L2",
    "Landsat_7": "LANDSAT/LE07/C02/T1_L2",
    "Landsat_8": "LANDSAT/LC08/C02/T1_L2",
    "Landsat_9": "LANDSAT/LC09/C02/T1_L2",
}

In [98]:
def get_times(collection_id, region):
    col = (
        ee.ImageCollection(collection_id)
        .filterBounds(region)
    )

    def extract_times(img):
        return ee.Feature(None, {
            "collection": collection_id,
            "image_id": img.id(),
            "start_time": ee.Date(img.get("system:time_start")).format("YYYY-MM-dd HH:mm:ss"),
            "end_time": ee.Date(img.get("system:time_end")).format("YYYY-MM-dd HH:mm:ss")
        })

    return col.map(extract_times)

In [99]:
all_features = []

for name, collection_id in landsat_collections.items():
    features = get_times(collection_id, buffered_region)
    all_features.append(features)

# Merge all results
merged = ee.FeatureCollection(all_features).flatten()


In [100]:
start_times, end_times = [], []

In [101]:
results = merged.getInfo()

for f in results["features"]:
    props = f["properties"]
    print(
        props["collection"],
        props["image_id"],
        props["start_time"],
        props["end_time"]
    )
    start_times.append(props["start_time"])
    end_times.append(props["end_time"])


LANDSAT/LT04/C02/T1_L2 LT04_012030_19821231 1982-12-31 14:55:17 1982-12-31 14:55:17
LANDSAT/LT04/C02/T1_L2 LT04_012031_19830201 1983-02-01 14:56:22 1983-02-01 14:56:22
LANDSAT/LT04/C02/T1_L2 LT04_012031_19890828 1989-08-28 15:00:35 1989-08-28 15:00:35
LANDSAT/LT05/C02/T1_L2 LT05_012030_19850129 1985-01-29 14:57:07 1985-01-29 14:57:07
LANDSAT/LT05/C02/T1_L2 LT05_012030_19870324 1987-03-24 14:48:55 1987-03-24 14:48:55
LANDSAT/LT05/C02/T1_L2 LT05_012030_19870730 1987-07-30 14:52:17 1987-07-30 14:52:17
LANDSAT/LT05/C02/T1_L2 LT05_012030_19870831 1987-08-31 14:53:12 1987-08-31 14:53:12
LANDSAT/LT05/C02/T1_L2 LT05_012030_19870916 1987-09-16 14:53:35 1987-09-16 14:53:35
LANDSAT/LT05/C02/T1_L2 LT05_012030_19871018 1987-10-18 14:54:13 1987-10-18 14:54:13
LANDSAT/LT05/C02/T1_L2 LT05_012030_19871205 1987-12-05 14:55:07 1987-12-05 14:55:07
LANDSAT/LT05/C02/T1_L2 LT05_012030_19871221 1987-12-21 14:55:21 1987-12-21 14:55:21
LANDSAT/LT05/C02/T1_L2 LT05_012030_19880106 1988-01-06 14:55:37 1988-01-06 1

In [102]:
import numpy as np

In [103]:
start_times = np.array(start_times).astype('datetime64')
end_times = np.array(end_times).astype('datetime64')

In [104]:
def min_time_of_day(datetime64_array):
    """
    Find the minimum time of day from a numpy array of datetime64 objects.

    Args:
        datetime64_array: A numpy array with dtype='datetime64[unit]'.

    Returns:
        The minimum time of day as a numpy.timedelta64 object.
    """
    # 1. Cast the array to 'datetime64[D]' (day resolution) to get the start of each day.
    #    This removes the time component, setting it to midnight (00:00:00) for each date.
    midnights = datetime64_array.astype('datetime64[D]')

    # 2. Subtract the start of the day from the original datetime to get the
    #    time of day (as a timedelta from midnight) for each element.
    times_of_day = datetime64_array - midnights

    # 3. Find the minimum value among the resulting timedeltas.
    minimum_time = times_of_day.min()

    return minimum_time

In [105]:
start_times[0:5]

array(['1982-12-31T14:55:17', '1983-02-01T14:56:22',
       '1989-08-28T15:00:35', '1985-01-29T14:57:07',
       '1987-03-24T14:48:55'], dtype='datetime64[s]')

In [107]:
# Convert to seconds since midnight
seconds_since_midnight = (
    start_times.astype('datetime64[s]')
    - start_times.astype('datetime64[D]')
).astype(int)

# Find minimum
min_seconds = seconds_since_midnight.min()

# Convert back to HH:MM:SS
hours = min_seconds // 3600
minutes = (min_seconds % 3600) // 60
seconds = min_seconds % 60

min_time_of_day = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

print(min_time_of_day)


13:03:44


In [108]:
# Find maximum
max_seconds = seconds_since_midnight.max()

# Convert back to HH:MM:SS
hours = max_seconds // 3600
minutes = (max_seconds % 3600) // 60
seconds = max_seconds % 60

max_time_of_day = f"{hours:02d}:{minutes:02d}:{seconds:02d}"

print(max_time_of_day)

15:31:56
